In [99]:
from pathlib import Path
from spacy.lang.en import English
from spacy.tokenizer import Span
from spacy.tokens import Doc

In [100]:
train_data_path = Path('./assets/sem-eval/TRAIN_FILE.TXT')
nlp = English()
Doc.set_extension("rel", default={})

In [128]:
data = list(train_data.open())
samples = []
relation_labels=set()
for index in range(0, len(data), 4):
    sample = {}
    sentence = "\"".join(data[index].split("\"")[1:-1])
    relation = data[index+1].split('(')[0]
    if(len(data[index+1].split('(')) > 1): # there are relationship
        first_entity = data[index+1].split('(')[1].split(',')[0]
    else:
        first_entity = None
        relation = "Other"
    e1_start_index = sentence.find("<e1>")
    e1_end_index = sentence.find("</e1>")-4
    sentence = sentence.replace("<e1>", "")
    sentence = sentence.replace("</e1>", "")
    
    e2_start_index = sentence.find("<e2>")
    e2_end_index = sentence.find("</e2>")-4
    sentence = sentence.replace("<e2>", "")
    sentence = sentence.replace("</e2>", "")
    
#     sample["e1"] = sample["sentence"][e1_start_index:e1_end_index]
#     sample["e2"] = sample["sentence"][e2_start_index:e2_end_index]
    
    doc= nlp(sentence)
    try:
        e1 = doc.char_span(e1_start_index, e1_end_index, label="e1")
        e2 = doc.char_span(e2_start_index, e2_end_index, label="e2")
        doc.set_ents([e1, e2])
    except:
        print("error")
        continue
    if(first_entity):
        doc._.rel = {
            (e1.start, e2.start): {
                relation: 1.0 if first_entity=='e1' else 0
            },
            (e2.start, e1.start): {
                relation: 1.0 if first_entity=='e2' else 0
            }
        }
    else:
        doc._.rel = {
            (e1.start, e2.start): {
                relation: 1
            },
            (e1.start, e2.start): {
                relation: 1
            }
        }
    relation_labels.add(relation)
    samples.append(doc)
    

error
error
error
error
error


In [129]:
relation_labels

{'Cause-Effect',
 'Component-Whole',
 'Content-Container',
 'Entity-Destination',
 'Entity-Origin',
 'Instrument-Agency',
 'Member-Collection',
 'Message-Topic',
 'Other',
 'Product-Producer'}

In [130]:
for doc in samples:
    for key in doc._.rel:
        for r in relation_labels:
            if(r not in doc._.rel[key]):
                doc._.rel[key][r] = 0

In [134]:
samples[2]._.rel

{(1, 7): {'Instrument-Agency': 0,
  'Entity-Destination': 0,
  'Component-Whole': 0,
  'Other': 0,
  'Member-Collection': 0,
  'Content-Container': 0,
  'Cause-Effect': 0,
  'Product-Producer': 0,
  'Entity-Origin': 0,
  'Message-Topic': 0},
 (7, 1): {'Instrument-Agency': 1.0,
  'Entity-Destination': 0,
  'Component-Whole': 0,
  'Other': 0,
  'Member-Collection': 0,
  'Content-Container': 0,
  'Cause-Effect': 0,
  'Product-Producer': 0,
  'Entity-Origin': 0,
  'Message-Topic': 0}}

In [52]:
nlp = English()
doc = nlp(samples[0]["sentence"])

In [54]:
for token in doc:
    print(token.text)

The
system
as
described
above
has
its
greatest
application
in
an
arrayed
<
e1
>
configuration</e1
>
of
antenna
<
e2
>
elements</e2
>
.
